In [19]:
import torch

import torch.nn as nn
from torch.utils import data
from transformers import RobertaTokenizer, RobertaModel

# bs64_lr3.0e-02_drop0_nlayer3_hiddim256

args = {
    'batch_size': 64,
    'lr': .0e-02,
    'hidden_dim': 256,
    'n_layers': 3,
    'bidirectional': True,
    'dropout': 0,
    'n_epochs': None,
    'b1': None,
    'b2': None,
    'weight_decay': None,
    'lr_decay': 0.7
}

In [3]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [6]:
import pandas as pd

df = pd.read_csv("../data/hydrated_data_ieee/final_data/shallow_processed.csv")
df

,Unnamed: 0,Unnamed: 0.1,tweet_id,text,retweet_count,favorite_count,possibly_sensitive,withheld_in_countries,user.id,user.name,...,user.listed_count,user.created_at,user.favourites_count,user.statuses_count,user.default_profile,user.default_profile_image,user.withheld_in_countries,date,sentiment_score,expanded_text
0,0,0,1.240728e+18,holy shit I hate every person in this video,70104,0,NaN,NaN,2.022756e+08,Typreme Kicks.👟❤️📈🤙🏽,...,9,Wed Oct 13 17:48:24 +0000 2010,89995,63405,True,False,NaN,Mar 19,-0.500000,holy shit I hate every person in this video
1,1,1,1.240728e+18,There you have it.\n\nProof that this is an in...,1662,0,NaN,NaN,3.524761e+08,Jake,...,3,Wed Aug 10 17:37:59 +0000 2011,9823,21872,False,False,NaN,Mar 19,0.000000,There you have it.\n\nProof that this is an in...
2,2,2,1.240728e+18,"Among other things, what this shows is they're...",6624,0,NaN,NaN,9.281008e+17,Fifty Pound Head,...,0,Wed Nov 08 03:24:12 +0000 2017,1651,1617,False,False,NaN,Mar 19,-0.062500,"Among other things, what this shows is they ar..."
3,3,3,1.240729e+18,this incompetent racist asshole,14445,0,False,NaN,1.585784e+07,Brad,...,2,Fri Aug 15 00:33:06 +0000 2008,8538,11553,False,False,NaN,Mar 19,-0.350000,this incompetent racist asshole
4,4,4,1.240729e+18,"Among other things, what this shows is they're...",6624,0,NaN,NaN,1.001945e+18,Pamela ( No DMs),...,3,Wed May 30 21:56:16 +0000 2018,190490,99423,True,False,NaN,Mar 19,-0.062500,"Among other things, what this shows is they ar..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52337,52337,52337,1.267310e+18,COVID-19 has a 2-14 day incubation period. If ...,17700,0,NaN,NaN,1.629978e+07,Andrea,...,87,Mon Sep 15 18:58:35 +0000 2008,18604,76072,False,False,NaN,Jun 01,0.000000,COVID-19 has a 2-14 day incubation period. If ...
52338,52338,52338,1.267311e+18,"First they burned down our economy, impoverish...",32770,0,NaN,NaN,1.063319e+18,YouAndIMAGA 🇺🇸,...,1,Fri Nov 16 06:32:07 +0000 2018,667,972,True,False,NaN,Jun 01,0.047222,"First they burned down our economy, impoverish..."
52339,52339,52339,1.267311e+18,Fears about COVID-19 can take an emotional tol...,2,0,NaN,NaN,2.229163e+09,Sharon Arnold,...,8,Wed Dec 04 02:37:56 +0000 2013,9246,2460,True,False,NaN,Jun 01,0.000000,Fears about COVID-19 can take an emotional tol...
52340,52340,52340,1.267311e+18,Not the seafood tho 😂,1,3,False,NaN,7.233305e+17,Key_🥀,...,0,Fri Apr 22 02:00:04 +0000 2016,3875,498,True,False,NaN,Jun 01,0.000000,Not the seafood tho face with tears of joy


In [7]:
tokenized = df["expanded_text"].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

In [8]:
def get_max_len(tokenized):
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    return max_len

max_len = get_max_len(tokenized)
padded = torch.tensor([i + [0] * (max_len - len(i)) for i in tokenized.values])

In [9]:
class Dataset(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, x):
        'Initialization'
        self.x = x

    def __len__(self):
        'Denotes the total number of samples'
        return self.x.shape[0]

    def __getitem__(self, index):
        'Generates one sample of data'
        x = self.x[index]

        return x

In [11]:
dataset = Dataset(padded)

In [13]:
dataset_loader = torch.utils.data.DataLoader(dataset,
                                             batch_size=args["batch_size"],
                                             shuffle=False,
                                             drop_last=False)

In [16]:
class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        # embedding_dim = bert.config.to_dict()['dim']
        embedding_dim = 768
    
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)

        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        attention_mask = text.masked_fill(text != 0, 1)
                
        with torch.no_grad():
            embedded = self.bert(text, attention_mask=attention_mask)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [20]:
bert = RobertaModel.from_pretrained('roberta-base')

model = BERTGRUSentiment(bert,
                         args['hidden_dim'],
                         3,
                         args['n_layers'],
                         args['bidirectional'],
                         args['dropout'])

In [22]:
model.load_state_dict(torch.load(f"../experiments/unprocessed_bs64_lr3.0e-02_drop0_nlayer3_hiddim256/bs64_lr3.0e-02_drop0_nlayer3_hiddim256.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
def get_prediction(data):
    pred = model(data)
    pred_prob = softmax(pred)
    pred_hard = torch.max(pred_prob, dim=1)
    return pred_hard

result = []
softmax = torch.nn.Softmax(dim=1)
for batch_idx, data in enumerate(dataset_loader):
    with torch.no_grad():
        result.append(get_prediction(data))

result = torch.cat(result)